In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling２D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import cv2
from PIL import Image
import os.path as osp
import os
from math import pi
import torch
import pickle
import scipy as sp
import pandas as pd
from sklearn import preprocessing
HOME = osp.expanduser("~")
HOME

2022-12-19 05:33:15.589119: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'/home/yy/y.shinozaki'

In [90]:
"""import emnist
images_train, labels_train = emnist.extract_training_samples('letters')
images_test, labels_test = emnist.extract_test_samples('letters')"""

In [43]:
for i in range(10200,10500):
    img_dst = images_test[i]
    Image.fromarray(img_dst.reshape(28,28)).show()

NameError: name 'images_test' is not defined

In [76]:
"""df_train = pd.DataFrame(images_train.reshape(124800,784))
df_test = pd.DataFrame(images_test.reshape(20800,784))
df_train_label = pd.DataFrame(labels_train.reshape(124800,))
df_test_label = pd.DataFrame(labels_test.reshape(20800,))
df_train.to_csv("EMNIST/raw_train_emnist_images.csv",index=False)
df_test.to_csv("EMNIST/raw_test_emnist_images.csv",index=False)
df_train_label.to_csv("EMNIST/raw_train_emnist_labels.csv",index=False)
df_test_label.to_csv("EMNIST/raw_test_emnist_labels.csv",index=False)"""

In [29]:
label_train = pd.read_csv("EMNIST/raw_train_emnist_labels.csv")

In [92]:
"""labels_train_base = np.array(pd.read_csv("EMNIST/raw_train_emnist_labels.csv"))
X_train_base = pd.read_csv("EMNIST/raw_train_emnist_images.csv")
print(X_train_base.shape)
print(labels_train_base.shape)"""

(124800, 784)
(124800, 1)


In [2]:
from tensorflow.keras.layers import Flatten

# MNIST 複数チャンネル用
# Kerasに付属の手書き数字画像データをダウンロード
labels_train_base = np.array(pd.read_csv("EMNIST/raw_train_emnist_labels.csv"))-1#.reshape(-1)
X_train_base = pd.read_csv("EMNIST/raw_train_emnist_images.csv")
labels_test = np.array(pd.read_csv("EMNIST/raw_test_emnist_labels.csv"))-1#.reshape(-1)
#labels_test = dataset_raw["y_test"]

# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2) 
# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き
X_test = np.array(pd.read_csv("EMNIST/raw_test_emnist_images.csv")).reshape(20800,28,28).astype('float32')
X_test /= 255
X_train = np.array(X_train).reshape(99840,28,28).astype('float32')
X_train /= 255
X_validation = np.array(X_validation).reshape(24960,28,28).astype('float32')
X_validation /= 255
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)    

print("X_train:",X_train.shape)
print("X_validation:",X_validation.shape)
print("X_test:",X_test.shape)
print("y_train:",y_train.shape)
print("y_validation:",y_validation.shape)
print("y_test:",y_test.shape)

# 学習率
rate       = 0.005

# Sequentialクラスを使ってモデルを準備する
model = Sequential()

# 隠れ層を追加
#model.add(Dense(32,kernel_regularizer=keras.regularizers.l2(rate),input_shape=(28, 28,1)))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())

# 出力層を追加
model.add(Dense(26, activation='softmax'))

# TensorFlowのモデルを構築
model.compile(optimizer=tf.optimizers.Adam(rate),
              loss='categorical_crossentropy', metrics=['categorical_crossentropy', 'accuracy'])

# Early stoppingを適用してフィッティング
log1 = model.fit(X_train, y_train, batch_size=208 ,epochs=20, verbose=True,
                #callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                #     min_delta=0, patience=10, 
                                                  #       verbose=1)],
                validation_data=(X_validation, y_validation))

# Test dataで予測を実行。
pred_test = np.argmax(model.predict(X_test), axis=-1)

validation = (pred_test == y_test.nonzero()[1])
size       = validation.size
size
correct    = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

X_train: (99840, 28, 28)
X_validation: (24960, 28, 28)
X_test: (20800, 28, 28)
y_train: (99840, 26)
y_validation: (24960, 26)
y_test: (20800, 26)


2022-12-19 05:33:54.354525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 05:33:54.355337: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/20
480/480 [==============================] - 39s 80ms/step - loss: 0.6838 - categorical_crossentropy: 0.6838 - accuracy: 0.7956 - val_loss: 0.4268 - val_categorical_crossentropy: 0.4268 - val_accuracy: 0.8687
Epoch 2/20
480/480 [==============================] - 39s 80ms/step - loss: 0.4246 - categorical_crossentropy: 0.4246 - accuracy: 0.8673 - val_loss: 0.3783 - val_categorical_crossentropy: 0.3783 - val_accuracy: 0.8828
Epoch 3/20
480/480 [==============================] - 39s 82ms/step - loss: 0.3779 - categorical_crossentropy: 0.3779 - accuracy: 0.8810 - val_loss: 0.3634 - val_categorical_crossentropy: 0.3634 - val_accuracy: 0.8861
Epoch 4/20
480/480 [==============================] - 39s 82ms/step - loss: 0.3455 - categorical_crossentropy: 0.3455 - accuracy: 0.8906 - val_loss: 0.3412 - val_categorical_crossentropy: 0.3412 - val_accuracy: 0.8923
Epoch 5/20
480/480 [==============================] - 39s 82ms/step - loss: 0.3236 - categorical_crossentropy: 0.3236 - accuracy

In [3]:
pd.DataFrame(log1.history).to_csv(HOME+"/研究/MNIST/EMNIST/emnist_raw0.005_log1.csv",index=False)
a = pd.read_csv(HOME+"/研究/MNIST/EMNIST/emnist_raw0.005_log1.csv")
a
# 0.001 18642/20800 correct (89.625%)
# 0.005 18366/20800 correct (88.298%)(88.995%)
#18837/20800 correct (90.562%))(最新版)

,loss,categorical_crossentropy,accuracy,val_loss,val_categorical_crossentropy,val_accuracy
0,0.683809,0.683809,0.795553,0.426850,0.426850,0.868670
1,0.424607,0.424607,0.867308,0.378258,0.378258,0.882812
2,0.377873,0.377873,0.881000,0.363405,0.363405,0.886098
3,0.345506,0.345506,0.890565,0.341202,0.341202,0.892268
4,0.323601,0.323601,0.896194,0.333289,0.333289,0.899159
5,0.306249,0.306249,0.901342,0.324103,0.324103,0.902364
6,0.292177,0.292177,0.904888,0.317013,0.317013,0.904006
7,0.281467,0.281467,0.907212,0.324098,0.324098,0.900881
8,0.270953,0.270953,0.910657,0.316516,0.316516,0.905369
9,0.261557,0.261557,0.912821,0.314622,0.314622,0.905288


In [175]:
"""#　データセットを準備
X_train_base = pd.read_csv("EMNIST/raw_train_emnist_images.csv")
X_test_base = pd.read_csv("EMNIST/raw_test_emnist_images.csv")

# ガウシアンでぼかす
import cv2

#　ぼかした後の画像をまとめるnumpy配列
#　訓練データ
#dataset["X_train_val_first_gauss"] = np.array([])
first_gauss_dic = {}
list1 = [[]]*124800
list2 = [[]]*20800
#X_train_base = np.array(X_train_base)
#X_test_base = np.array(X_test_base)
#X_train_base = X_train_base.reshape(124800,28,28)
#X_test_base = X_test_base.reshape(20800,28,28)
X_train_base = np.array(X_train_base,dtype=np.uint8)
for i in range(X_train_base.shape[0]):

    # Spatial filtering
    dst_kari = X_train_base[i].reshape(28,28)
    dst = cv2.GaussianBlur(dst_kari, ksize=(3, 3), sigmaX=0.5)

    # output
    # 結果を出力
    list1[i] = (dst.reshape(-1,784)[0])
list1 = pd.DataFrame(list1)
list1.to_csv("EMNIST/first_gauss_train.csv",index=False)

#　テストデータ
X_test_base = np.array(X_test_base,dtype=np.uint8)
for i in range(X_test_base.shape[0]):
    
    # convert grayscale
    # グレースケール変換
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Spatial filtering
    dst_kari = X_test_base[i].reshape(28,28)
    dst = cv2.GaussianBlur(dst_kari, ksize=(3, 3), sigmaX=0.5)

    # output
    # 結果を出力
    list2[i] = (dst.reshape(-1,784)[0])

list2 = pd.DataFrame(list2)
list2.to_csv("EMNIST/first_gauss_test.csv",index=False)"""

In [187]:
#　１回目のガウスぼかしの結果を確認
first_gauss_train = pd.read_csv("EMNIST/first_gauss_test.csv")
for i in range(10):
    img_dst = np.array(first_gauss_train)[i].astype(np.int8)
    Image.fromarray(img_dst.reshape(28,28)).show()

(高速で画像を保存するのに参考になった)[ https://qiita.com/dzbt_dzbt/items/a54f209cca6adeedb96f ]

In [27]:
# 画像を微分する

import cv2
import numpy as np
import time


# カーネル（水平、垂直方向の輪郭検出用）
kernel_y = np.array([[0, -1, 0],
                         [0, 0, 0],
                         [0, 1, 0]])

kernel_x = np.array([[0, 0, 0],
                        [-1, 0, 1],
                        [0, 0, 0]])

first_gauss_train_x = np.array(pd.read_csv("EMNIST/first_gauss_train.csv"),np.int16)
first_gauss_train_y = np.array(pd.read_csv("EMNIST/first_gauss_train.csv"),np.int16)
first_gauss_train_dst = np.array(pd.read_csv("EMNIST/first_gauss_train.csv"),np.int16)

diff_X_train = np.zeros((124800,784))
diff_y_train = np.zeros((124800,784))
diff_X_test = np.zeros((20800,784))
diff_y_test = np.zeros((20800,784))

#　訓練画像を微分

for i in range(first_gauss_train_x.shape[0]):
    if i%5000==0:
        print("{}まで終了".format(i))
        now = time.strftime('%Y/%m/%d %H:%M:%S')
        print(now)

    # 方法2
    gray_x = cv2.filter2D(first_gauss_train_x[i].reshape(28,28), cv2.CV_64F, kernel_x).reshape(-1,784)
    gray_y = cv2.filter2D(first_gauss_train_y[i].reshape(28,28), cv2.CV_64F, kernel_y).reshape(-1,784)
    #dst = gray_x ** 2 + gray_y ** 2
    #list1[i] = (dst.reshape(-1,784)[0])
    diff_X_train[i] = gray_x.reshape(-1,784)[0]
    diff_y_train[i] = gray_y.reshape(-1,784)[0]
    #first_gauss_train_dst[i] = (np.float32(gray_x)**2 + np.float32(gray_y)**2).reshape(-1,784)[0]
    #first_gauss_train_dst[i] = dst.reshape(-1,784)[0]
#first_gauss_train_x = np.array(first_gauss_train_x,dtype=np.float16)
#first_gauss_train_y = np.array(first_gauss_train_y,dtype=np.float16)
dst = np.float32(diff_X_train)**2+np.float32(diff_y_train)**2
# 結果を保存
diff_train_x_header = np.array(pd.DataFrame(diff_X_train).columns.tolist())
diff_train_y_header = np.array(pd.DataFrame(diff_y_train).columns.tolist())
diff_train_dst_header = np.array(pd.DataFrame(dst).columns.tolist())
np.savez_compressed('EMNIST/X_diff_train_emnist', diff_train_x_header, diff_X_train)
np.savez_compressed('EMNIST/y_diff_train_emnist', diff_train_y_header, diff_y_train)
np.savez_compressed('EMNIST/dst_train_emnist', diff_train_dst_header, dst)

#いらないデータを消す
del diff_X_train, diff_y_train, dst


##################################################################
#　テスト画像を微分
first_gauss_test_x = np.array(pd.read_csv("EMNIST/first_gauss_test.csv"),np.int16)
first_gauss_test_y = np.array(pd.read_csv("EMNIST/first_gauss_test.csv"),np.int16)
first_gauss_test_dst = np.array(pd.read_csv("EMNIST/first_gauss_test.csv"),np.int16)


for i in range(first_gauss_test_x.shape[0]):
    if i%5000==0:
        print("{}まで終了".format(i))
        now = time.strftime('%Y/%m/%d %H:%M:%S')
        print(now)

    # 方法2
    gray_x = cv2.filter2D(first_gauss_test_x[i].reshape(28,28), cv2.CV_64F, kernel_x)
    gray_y = cv2.filter2D(first_gauss_test_y[i].reshape(28,28), cv2.CV_64F, kernel_y)
    
    diff_X_test[i] = gray_x.reshape(-1,784)[0]
    diff_y_test[i] = gray_y.reshape(-1,784)[0]
    #first_gauss_test_dst[i] = (np.float32(gray_x)**2 + np.float32(gray_y)**2).reshape(-1,784)[0]
#first_gauss_test_x = np.array(first_gauss_test_x,dtype=np.float16)
#first_gauss_test_y = np.array(first_gauss_test_y,dtype=np.float16)
dst = np.float32(diff_X_test) ** 2 + np.float32(diff_y_test) ** 2
diff_test_x_header = np.array(pd.DataFrame(diff_X_test).columns.tolist())
diff_test_y_header = np.array(pd.DataFrame(diff_y_test).columns.tolist())
diff_test_dst_header = np.array(pd.DataFrame(dst).columns.tolist())
np.savez_compressed('EMNIST/X_diff_test_emnist', diff_test_x_header, diff_X_test)
np.savez_compressed('EMNIST/y_diff_test_emnist', diff_test_y_header, diff_y_test)
np.savez_compressed('EMNIST/dst_test_emnist', diff_test_dst_header, dst) 

0まで終了
2022/11/28 03:03:22
5000まで終了
2022/11/28 03:03:22
10000まで終了
2022/11/28 03:03:22
15000まで終了
2022/11/28 03:03:22
20000まで終了
2022/11/28 03:03:22
25000まで終了
2022/11/28 03:03:22
30000まで終了
2022/11/28 03:03:22
35000まで終了
2022/11/28 03:03:22
40000まで終了
2022/11/28 03:03:22
45000まで終了
2022/11/28 03:03:22
50000まで終了
2022/11/28 03:03:22
55000まで終了
2022/11/28 03:03:22
60000まで終了
2022/11/28 03:03:22
65000まで終了
2022/11/28 03:03:22
70000まで終了
2022/11/28 03:03:22
75000まで終了
2022/11/28 03:03:23
80000まで終了
2022/11/28 03:03:23
85000まで終了
2022/11/28 03:03:23
90000まで終了
2022/11/28 03:03:23
95000まで終了
2022/11/28 03:03:23
100000まで終了
2022/11/28 03:03:23
105000まで終了
2022/11/28 03:03:23
110000まで終了
2022/11/28 03:03:23
115000まで終了
2022/11/28 03:03:23
120000まで終了
2022/11/28 03:03:23


OSError: [Errno 5] Input/output error

In [4]:
X_diff_train = np.load("EMNIST/X_diff_train_emnist.npz", allow_pickle=True)
y_diff_train = np.load("EMNIST/y_diff_train_emnist.npz", allow_pickle=True) 
X_diff_test = np.load("EMNIST/X_diff_test_emnist.npz", allow_pickle=True)
y_diff_test = np.load("EMNIST/y_diff_test_emnist.npz", allow_pickle=True)
dst_train = np.load("EMNIST/dst_train_emnist.npz", allow_pickle=True)
dst_test = np.load("EMNIST/dst_test_emnist.npz", allow_pickle=True)
# npz形式に複数のNumpy配列を保存した場合、'arr_0', 'arr_1'…の順に取り出せる
np_X_diff_train = np.array(pd.DataFrame(X_diff_train['arr_1'], columns=X_diff_train['arr_0']))
np_y_diff_train = np.array(pd.DataFrame(y_diff_train['arr_1'], columns=y_diff_train['arr_0']))
np_X_diff_test = np.array(pd.DataFrame(X_diff_test['arr_1'], columns=X_diff_test['arr_0']))
np_y_diff_test = np.array(pd.DataFrame(y_diff_test['arr_1'], columns=y_diff_test['arr_0']))
np_dst_train = np.array(pd.DataFrame(dst_train['arr_1'], columns=dst_train['arr_0']))
np_dst_test = np.array(pd.DataFrame(dst_test['arr_1'], columns=dst_test['arr_0']))

In [ ]:
#　１回目の微分の結果を確認
for i in range(10):
    img_X_diff = np_X_diff_test[i]
    #img_y_diff = np_dst_test[i]**0.5
    #img_X_diff = np_X_diff_test[i]
    Image.fromarray(img_X_diff.reshape(28,28)).show()
print("\n")
for i in range(10):
    img_y_diff = np_y_diff_test[i]
    Image.fromarray(img_y_diff.reshape(28,28)).show()

---
フーリエ記述子計算

In [5]:
#　フーリエ記述子計算

from math import pi

# epsilon
ep = 10 ** -12

# Q = 4とする
Q = 4

diff_train_image = np_X_diff_train - np_y_diff_train*1j
diff_test_image = np_X_diff_test - np_y_diff_test*1j

sqrt_train = (np_dst_train+ep) ** 0.5
sqrt_test = (np_dst_test+ep) ** 0.5
train_four_coeff = {}
test_four_coeff = {}

#　フーリエ記述子導出
for i in range(Q+1):
    dq_train = sqrt_train/(2*pi) * (diff_train_image/sqrt_train) ** i
    dq_test = sqrt_test/(2*pi) * (diff_test_image/sqrt_test) ** i
    train_four_coeff_real = dq_train.real
    train_four_coeff_imag = dq_train.imag
    test_four_coeff_real = dq_test.real
    test_four_coeff_imag = dq_test.imag
    if i==0:
        np_train_four_coeff = np.float32(train_four_coeff_real)
        #np_train_four_coeff = np.append(np_train_four_coeff,train_four_coeff_imag,axis=0)
        np_test_four_coeff = np.float32(test_four_coeff_real)
        #np_test_four_coeff = np.append(np_test_four_coeff,test_four_coeff_imag,axis=0)
    else:
        #np_train_four_coeff = np.append(np_train_four_coeff,train_four_coeff_real,axis=0)
        np_train_four_coeff = np.append(np_train_four_coeff,np.float32(train_four_coeff_real),axis=0)
        np_train_four_coeff = np.append(np_train_four_coeff,np.float32(train_four_coeff_imag),axis=0)
        #np_train_four_coeff = np.float32(np_train_four_coeff.reshape(2, 124800,784))
        np_test_four_coeff = np.append(np_test_four_coeff,np.float32(test_four_coeff_real),axis=0)
        np_test_four_coeff = np.append(np_test_four_coeff,np.float32(test_four_coeff_imag),axis=0)
        #np_test_four_coeff = np.float32(np_test_four_coeff.reshape(2, 20800,784))
np_train_four_coeff = np_train_four_coeff.reshape(9,124800 ,784)
np_test_four_coeff = np_test_four_coeff.reshape(9, 20800,784)
train_header = np.array(pd.DataFrame(np_train_four_coeff[0]).columns.tolist())
test_header = np.array(pd.DataFrame(np_test_four_coeff[0]).columns.tolist())
#np.savez_compressed('EMNIST/train_four_coeff', train_header, np_train_four_coeff)
#np.savez_compressed('EMNIST/test_four_coeff', test_header, np_test_four_coeff)
"""
print(np_train_four_coeff.shape)
print(np_test_four_coeff.shape)
  train_four_coeff["{}_real".format(i)] = train_four_coeff_real
    train_four_coeff["{}_imag".format(i)] = train_four_coeff_imag
    test_four_coeff["{}_real".format(i)] = test_four_coeff_real
    test_four_coeff["{}_imag".format(i)] = test_four_coeff_imag"""

"""#　データを保存
train_header = np.array(pd.DataFrame(train_four_coeff).columns.tolist())
test_header = np.array(pd.DataFrame(test_four_coeff).columns.tolist())
np.savez('EMNIST/train_four_coeff', train_header, np_train_four_coeff)
np.savez('EMNIST/test_four_coeff', test_header, np_test_four_coeff)"""

"#\u3000データを保存\ntrain_header = np.array(pd.DataFrame(train_four_coeff).columns.tolist())\ntest_header = np.array(pd.DataFrame(test_four_coeff).columns.tolist())\nnp.savez('EMNIST/train_four_coeff', train_header, np_train_four_coeff)\nnp.savez('EMNIST/test_four_coeff', test_header, np_test_four_coeff)"

2回目のガウスぼかし

In [6]:
# ガウシアンでぼかす
import cv2
# load image (grayscale)
# 入力画像を読み込み
#img_train = dataset["X_train"]
#img_test = dataset["X_test"]

#　ぼかした後の画像をまとめるnumpy配列
#　訓練データ
Q = 4
#np_train_four_coeff = np_train_four_coeff.reshape(9,124800 ,784)
#np_test_four_coeff = np_test_four_coeff.reshape(9, 20800,784)
second_gauss_train = np.zeros((9,124800,784))
second_gauss_test = np.zeros((9,20800,784))
for i in range(2*Q+1):
    kari_gauss_train = cv2.GaussianBlur(np_train_four_coeff[i].reshape(124800,28,28), ksize=(3, 3), sigmaX=0.3)
    second_gauss_train[i] = kari_gauss_train.reshape(124800 ,784)
    kari_gauss_test = cv2.GaussianBlur(np_test_four_coeff[i].reshape(20800,28,28), ksize=(3, 3), sigmaX=0.3)
    second_gauss_test[i] = kari_gauss_test.reshape(20800,784)

ニューラルネットワーク構築

In [7]:
#　データセットを準備(9層)
stand_second_gauss_train = np.zeros((9,124800,784))
stand_second_gauss_test = np.zeros((9,20800,784))
for i in range(2*Q+1):
    stand_second_gauss_train[i] = sp.stats.zscore(second_gauss_train[i],axis=None)
    #stand_second_gauss_train[i] = preprocessing.minmax_scale(second_gauss_train[i],axis=None)
    stand_second_gauss_test[i] = sp.stats.zscore(second_gauss_test[i],axis=None)
    #stand_second_gauss_test[i] = preprocessing.minmax_scale(second_gauss_test[i],axis=None)
X_train_base = np.stack((stand_second_gauss_train[0].reshape(124800,28,28),stand_second_gauss_train[1].reshape(124800,28,28),
                  stand_second_gauss_train[2].reshape(124800,28,28),stand_second_gauss_train[3].reshape(124800,28,28),
                  stand_second_gauss_train[4].reshape(124800,28,28),stand_second_gauss_train[5].reshape(124800,28,28),
                  stand_second_gauss_train[6].reshape(124800,28,28),stand_second_gauss_train[7].reshape(124800,28,28),
                  stand_second_gauss_train[8].reshape(124800,28,28)),axis=3)
X_test = np.stack((stand_second_gauss_test[0].reshape(20800,28,28),stand_second_gauss_test[1].reshape(20800,28,28),
                  stand_second_gauss_test[2].reshape(20800,28,28),stand_second_gauss_test[3].reshape(20800,28,28),
                  stand_second_gauss_test[4].reshape(20800,28,28),stand_second_gauss_test[5].reshape(20800,28,28),
                  stand_second_gauss_test[6].reshape(20800,28,28),stand_second_gauss_test[7].reshape(20800,28,28),
                  stand_second_gauss_test[8].reshape(20800,28,28)),axis=3)

In [7]:
#　データセットを準備(絶対値の5層)
stand_second_gauss_train = np.zeros((5,124800,784))
stand_second_gauss_test = np.zeros((5,20800,784))
for i in range(Q+1):
    if i==0:
        stand_second_gauss_train[i] = sp.stats.zscore(second_gauss_train[i],axis=None)
        #stand_second_gauss_train[i] = preprocessing.minmax_scale(second_gauss_train[i],axis=None)
        stand_second_gauss_test[i] = sp.stats.zscore(second_gauss_test[i],axis=None)
        #stand_second_gauss_test[i] = preprocessing.minmax_scale(second_gauss_test[i],axis=None)
    else:
        stand_second_gauss_train[i] = sp.stats.zscore((second_gauss_train[2*i-1]**2+second_gauss_train[2*i]**2)**0.5,axis=None)
        #stand_second_gauss_train[i] = preprocessing.minmax_scale(second_gauss_train[i],axis=None)
        stand_second_gauss_test[i] = sp.stats.zscore((second_gauss_test[2*i-1]**2+second_gauss_test[2*i]**2)**0.5,axis=None)
        #stand_second_gauss_test[i] = preprocessing.minmax_scale(second_gauss_test[i],axis=None)
        
X_train_base = np.stack((stand_second_gauss_train[0].reshape(124800,28,28),
                         stand_second_gauss_train[1].reshape(124800,28,28),stand_second_gauss_train[2].reshape(124800,28,28),
                         stand_second_gauss_train[3].reshape(124800,28,28),stand_second_gauss_train[4].reshape(124800,28,28))
                       ,axis=3)
X_test = np.stack((stand_second_gauss_test[0].reshape(20800,28,28),
                   stand_second_gauss_test[1].reshape(20800,28,28),stand_second_gauss_test[2].reshape(20800,28,28),
                   stand_second_gauss_test[3].reshape(20800,28,28),stand_second_gauss_test[4].reshape(20800,28,28))
                   ,axis=3)

In [30]:
X_test.shape

(20800, 28, 28, 5)

In [8]:
#　ニューラルネットワーク構築

from tensorflow.keras.layers import Flatten

# MNIST 複数チャンネル用
# Kerasに付属の手書き数字画像データをダウンロード
labels_train_base = (np.array(pd.read_csv("EMNIST/raw_train_emnist_labels.csv"))-1).reshape(-1)
labels_test = np.array(pd.read_csv("EMNIST/raw_test_emnist_labels.csv")-1).reshape(-1)
#X_train_base = X_train.transpose(0,2,3,1)

# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2) 
# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き

#X_test = X_test.transpose(3,1,2,0)
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)    

print("X_train:",X_train.shape)
print("X_validation:",X_validation.shape)
print("X_test:",X_test.shape)
print("y_train:",y_train.shape)
print("y_validation:",y_validation.shape)
print("y_test:",y_test.shape)

y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)

# 学習率
rate       = 0.005

# Sequentialクラスを使ってモデルを準備する
model = Sequential()

# 隠れ層を追加
#model.add(Dense(32,kernel_regularizer=keras.regularizers.l2(0.01),input_shape=(28, 28, 9)))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 9)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())

# 出力層を追加
model.add(Dense(26, activation='softmax'))

# TensorFlowのモデルを構築
model.compile(optimizer=tf.optimizers.Adam(rate),
              loss='categorical_crossentropy', metrics=['categorical_crossentropy', 'accuracy'])

# Early stoppingを適用してフィッティング
log2 = model.fit(X_train, y_train, batch_size=208 ,epochs=20, verbose=True,
                #callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', 
                  #                                   min_delta=0, patience=10, 
                    #                                     verbose=1)],
                validation_data=(X_validation, y_validation))

# Test dataで予測を実行。
pred_test = np.argmax(model.predict(X_test), axis=-1)

validation = (pred_test == labels_test)
size       = validation.size
size
correct    = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

X_train: (99840, 28, 28, 9)
X_validation: (24960, 28, 28, 9)
X_test: (20800, 28, 28, 9)
y_train: (99840, 26)
y_validation: (24960, 26)
y_test: (20800, 26)
Epoch 1/20
480/480 [==============================] - 34s 70ms/step - loss: 0.8972 - categorical_crossentropy: 0.8972 - accuracy: 0.7796 - val_loss: 0.4749 - val_categorical_crossentropy: 0.4749 - val_accuracy: 0.8512
Epoch 2/20
480/480 [==============================] - 36s 74ms/step - loss: 0.4603 - categorical_crossentropy: 0.4603 - accuracy: 0.8549 - val_loss: 0.4111 - val_categorical_crossentropy: 0.4111 - val_accuracy: 0.8731
Epoch 3/20
480/480 [==============================] - 37s 77ms/step - loss: 0.4089 - categorical_crossentropy: 0.4089 - accuracy: 0.8704 - val_loss: 0.3932 - val_categorical_crossentropy: 0.3932 - val_accuracy: 0.8762
Epoch 4/20
480/480 [==============================] - 36s 75ms/step - loss: 0.3828 - categorical_crossentropy: 0.3828 - accuracy: 0.8776 - val_loss: 0.3986 - val_categorical_crossentropy: 0.3

In [ ]:
#18204/20800 correct (87.519%)

#18625/20800 correct (89.543%)
#18500/20800 correct (88.942%)（最新）

In [9]:
pd.DataFrame(log2.history).to_csv(HOME+"/研究/MNIST/EMNIST/emnist_four_log2.csv",index=False)
a = pd.read_csv(HOME+"/研究/MNIST/EMNIST/emnist_four_log.csv")
a

,loss,categorical_crossentropy,accuracy,val_loss,val_categorical_crossentropy,val_accuracy
0,0.809151,0.767064,0.788502,0.461215,0.446753,0.858534
1,0.437936,0.426494,0.866236,0.408913,0.400079,0.870873
2,0.385985,0.377663,0.880679,0.393130,0.386296,0.877845
3,0.352047,0.344853,0.888972,0.384164,0.378126,0.880248
4,0.331163,0.324723,0.894581,0.371789,0.365958,0.886498
5,0.314768,0.308753,0.898397,0.353150,0.347522,0.890625
6,0.303174,0.297215,0.901352,0.356841,0.351166,0.890705
7,0.286729,0.281044,0.905078,0.354130,0.348955,0.889944
8,0.273116,0.267809,0.909555,0.350527,0.345812,0.891186
9,0.267004,0.261733,0.912250,0.355069,0.349834,0.889784
